In [1]:
# %load_ext autoreload
# %autoreload 2
from openmsimodel.db.open_db import OpenDB
import json

In [2]:
db = OpenDB('GEMD', '/home/arachid1/.private/gemd_db.json','/srv/hemi01-j01/openmsimodel/openmsimodel/db/queries_log')
model_id = 8

In [3]:
query = f""" SELECT * FROM GEMDObject WHERE model_id={model_id} AND gemd_type='measurement_run' """
_all = db.gemd_db.execute_query(query)
db.print_and_dump(_all, query, "_all", dump=False)

                                      uid  model_id        gemd_type   
0    123d8868-0ff4-4089-939f-775b8a24ea91         8  measurement_run  \
1    1377acb0-5c8b-4e09-bd06-e57e1f3a6953         8  measurement_run   
2    139d52a9-d720-48bd-8759-1f01fbf2782a         8  measurement_run   
3    141d2cca-f7ae-451c-aded-fdabec78613a         8  measurement_run   
4    14a08c04-f3b9-4352-8b3e-bfd5e89766d6         8  measurement_run   
..                                    ...       ...              ...   
180  2fa8af00-58d7-4a16-8524-83dc36047e89         8  measurement_run   
181  2ffee34b-adad-4831-8500-87aac0934f2d         8  measurement_run   
182  97d25757-65e4-4fb9-adf4-537a684714bc         8  measurement_run   
183  98f66b85-e270-42ec-bece-9865783ce0e7         8  measurement_run   
184  faa181eb-2955-402f-b1df-70262fae9995         8  measurement_run   

                                               context  
0    {"conditions": [], "file_links": [], "material...  
1    {"conditions": [

,uid,model_id,gemd_type,context
0,123d8868-0ff4-4089-939f-775b8a24ea91,8,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
1,1377acb0-5c8b-4e09-bd06-e57e1f3a6953,8,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
2,139d52a9-d720-48bd-8759-1f01fbf2782a,8,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
3,141d2cca-f7ae-451c-aded-fdabec78613a,8,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
4,14a08c04-f3b9-4352-8b3e-bfd5e89766d6,8,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
...,...,...,...,...
180,2fa8af00-58d7-4a16-8524-83dc36047e89,8,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
181,2ffee34b-adad-4831-8500-87aac0934f2d,8,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
182,97d25757-65e4-4fb9-adf4-537a684714bc,8,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
183,98f66b85-e270-42ec-bece-9865783ce0e7,8,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."


In [4]:
c = _all.iloc[1]['context']
print(json.dumps(json.loads(c), indent=3))

{
   "conditions": [],
   "file_links": [],
   "material": {
      "id": "d771ec10-420c-425d-9994-dc73fbebae22",
      "scope": "auto",
      "type": "link_by_uid"
   },
   "name": "XRD charact. for Alloy (AAB11-VAM-A) (T03)",
   "notes": null,
   "parameters": [],
   "properties": [
      {
         "file_links": [],
         "name": "Phase",
         "notes": "",
         "origin": "predicted",
         "template": null,
         "type": "property",
         "value": {
            "category": "FCC",
            "type": "nominal_categorical"
         }
      },
      {
         "file_links": [],
         "name": "Lattice Parameter",
         "notes": "",
         "origin": "predicted",
         "template": null,
         "type": "property",
         "value": {
            "nominal": 3.5792705228627013,
            "type": "nominal_real",
            "units": "centimeter"
         }
      },
      {
         "file_links": [],
         "name": "Hardness, HV",
         "notes": "",
     

a couple of invalid jsons that need fixing

In [5]:
query = f""" SELECT *
FROM GEMDObject 
WHERE model_id=8 and isjson(context)=0
"""
test = db.gemd_db.execute_query(query)
db.print_and_dump(test, query, "test")

                                    uid  model_id        gemd_type   
0  afab929b-397e-42fc-9d90-ddc90c7f23a9         8  measurement_run  \
1  1945b72a-dce8-4dd3-bdc7-ac84573896dd         8  measurement_run   

                                             context  
0  {"conditions": [], "file_links": [], "material...  
1  {"conditions": [], "file_links": [], "material...  


,uid,model_id,gemd_type,context
0,afab929b-397e-42fc-9d90-ddc90c7f23a9,8,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
1,1945b72a-dce8-4dd3-bdc7-ac84573896dd,8,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."


query to access all measurements

In [14]:
# query = f""" SELECT i.uid, OPENJSON(json_query(i.context, '$.properties') as properties

# value [nvarchar](32) case when type = 'nominal_categorical' then '$.value.category' 
#                 when type = 'nominal_real' then '$.value.nominal' 
#                 when type = 'nominal_integer' then '$.value.nominal' 
#                 when type = 'nominal_composition' then '$'
#                 else NULL 
query = f""" SELECT i.uid, properties.name, properties.type
        FROM gemdobject i 
        CROSS APPLY openjson(json_query(i.context, '$.properties')) 
        WITH 
        (name VARCHAR(50) '$.name', 
        type [nvarchar](32) '$.value.type',
        
        end)
        AS properties
        WHERE i.model_id={model_id} AND i.gemd_type='measurement_run' AND isjson(i.context)!=0"""
result = db.gemd_db.execute_query(query)
db.print_and_dump(result, query, "result")

OperationalError: (pymssql._pymssql.OperationalError) (156, b"Incorrect syntax near the keyword 'end'.DB-Lib error message 20018, severity 15:\nGeneral SQL Server error: Check messages from the SQL Server\n")
[SQL:  SELECT i.uid, properties.name, properties.type
        FROM gemdobject i 
        CROSS APPLY openjson(json_query(i.context, '$.properties')) 
        WITH 
        (name VARCHAR(50) '$.name', 
        type [nvarchar](32) '$.value.type',
        
        end)
        AS properties
        WHERE i.model_id=8 AND i.gemd_type='measurement_run' AND isjson(i.context)!=0]
(Background on this error at: http://sqlalche.me/e/e3q8)

TODO: further tailoring of query to specific measurements of interest for Bayesian Optimization